### old russian dictionary

In [1]:
from bs4 import BeautifulSoup
import csv
import numpy as np
import pandas as pd
import pickle
import requests
import re
from tqdm import tqdm_notebook
from selenium import webdriver
from selenium.webdriver.common.keys import Keys 
import time
import urllib
from selenium.webdriver import Chrome
tqdm_notebook().pandas()

pd.options.display.max_rows = 1000

In [20]:
def get_soup(url, simple_encoding=False):
    req = requests.get(url)
    if not simple_encoding:
        req.encoding = 'utf-8'
    soup = BeautifulSoup(req.text, 'html.parser')
    return soup

In [47]:
i = 0
url = 'https://rus-old-russian-dict.slovaronline.com/2-%D0%B0%D0%B1%D0%B8%D1%A5'
df = list()

with open(f"dict_11_14.csv", "w", encoding='utf-8') as csv_file:
    writer = csv.writer(csv_file, delimiter='\t')    
    while url:
        soup = get_soup(url)
        word = soup.find("strong").text
        full_text = soup.find("blockquote").text.strip().replace("\t", " ")
        writer.writerow((word, full_text, url))
        df.append({"word": word, "text": full_text, "url": url})
        i += 1
        url = soup.find("link", {"rel": "next"})['href']


In [2]:
df = pd.read_csv("dict_11_14_0.csv", names=["word", "text", "next_url"], sep="\t")

In [3]:
len(df)

12053

In [4]:
# with open("temp.pickle", "rb") as f:
#     df = pickle.load(f)

In [121]:
# shortening described in the book
shorts = """вин. — винительный (падеж)
вм. — вместо
возвр. — возвратный
вопр. — вопросительный
воскл. — восклицательный
вр. — время 
выделит. — выделительный 
гадат.— гадательная (приписка) 
геогр. — географический 
гл. — глагол 
грен. — греческий 
дат. — дательный (падеж) 
дв. — двойственное число 
действ. — действительный залог 
доп. — дополнение 
дополнит. — дополнительный 
др. — другой 
ед. — единственное число 
жен. — женский род 
зд. — здесь 
зная.— значение 
изд.— издание 
изъявит.— изъявительное (наклонение)
изъяснит.— изъяснительный
им. — имя 
имен. — именительный (падеж)
инф. — инфинитив
колин. — количественный
конкр. — конкретный
кратк. — краткий
л. — лицо 
ласкат. — ласкательный
личн. — личный
межд. — междометие
мест. — местоимение
местн. — местный (падеж)
мн. — множественное число
муж. — мужской род
модальн. — модальный
накл.— наклонение
нар. — наречие
наст. — настоящее время 
начинат. — начинательный
неопр. — неопределенный
нескл. — несклоняемый
образн. — образное (употребление)
Образн. — образное (употребление)
ок. — около 
определит. — определительный
отвл. — отвлеченный
отн. — относящийся 
относит. — относительный
отриц. — отрицательный
п. — падеж 
перен. — переносное (употребление) 
перечислит. — перечислительный 
побудит.— побудительный 
повелит.— повелительное (наклонение) 
подчинит.— подчинительное (наклонение) 
поздн. — поздний (список) 
полн. — полный 
пор. — порядковый 
пр. — прилагательное 
прав. — правильно 
предик. — предикатив 
предл. — предлог 
предлож. — предложение 
предположит. — предположительный 
придат.— придаточное (предложение) 
прилож. — приложение 
присоединит. — присоединительный 
притяж. — притяжательный 
прич. — причастие 
причин. — причинный 
прозе. — прозвище 
противит. — противительный 
противоп. — противоположное 
прош. — прошедшее время 
разделит. — разделительный 
род. — родительный (падеж) 
рук. — рукопись 
с. — существительное 
собир. — собирательный 
собств. — собственный 
соедин. — соединительный 
сопостав. — сопоставительный 
сослагат.— сослагательное (наклонение) 
сост. — состав 
соч. — сочетание 
сочин. — сочинительный 
союз. — союз
ср. — сравни 
сравн. — сравнительный 
среди. — средний род 
степ. — степень 
страд. — страдательный залог 
твор. — творительный (падеж) 
указат. — указательный 
уменым. — уменьшительный 
уничижит.— уничижительный
употр.— употребление
усилит.— усилительны
уcл. — условный
уступ. — уступительный 
ч. — частица
част. — частица
числ. — слово с числовым значением"""

shortenings = dict(tuple(y.strip() for y in x.split("—")) for x in shorts.split("\n"))

In [225]:
def parse_raw_text(s):    
    result = dict()
    bracked_num = re.search(r"\(.*?\d+.*?\),?", s)

    if not bracked_num:
        sm = re.search(r"\b[св]м\.", s)
        if sm:
            point = sm.span()[0]
            result["dict_word"] = s[:point].strip()
            return result
        rest = s
    else:
        result["dict_word"] = s[:bracked_num.span()[0]].strip()
        num = re.search(r"\d+", bracked_num[0])
        if num:
            result["quantity"] = int(num[0])
        rest = s[bracked_num.span()[1]:].strip()
    try:
        column_ind = rest.index(":")
    except ValueError:
        return result
    result["leftovers"] = rest[column_ind + 1:].strip()
    to_check = rest[:column_ind].split()
    for part in to_check:
#         print(part)
        if part.endswith(",") and ("1." not in part):
            if not part[:-1].isupper():
                result.setdefault("definition", list()).append(part)
        elif part.isupper() and (len(part) > 2):
             result.setdefault("alt", list()).append(part)
        elif "1." in part:
#             print("has 1.")
            halfs = part.split("1.")
            if halfs[0] in shortenings:
                result.setdefault("info", list()).append(halfs[0])
            result.setdefault("definition", list()).append(halfs[1])
#         elif part.startswith("-") or part.startswith("(-"):
        elif re.fullmatch(r"(?:\(?\-.+|˫?[АѦѢИЫ])", part):
#             print("has -")
            result.setdefault("alt", list()).append(part)
        elif part in shortenings:
#             print("short")
            result.setdefault("info", list()).append(part)
        else:
#             print("def")
            result.setdefault("definition", list()).append(part)
    result["definition"] = " ".join(result["definition"])
    return result

In [227]:
res = df.text.progress_apply(parse_raw_text)

In [228]:
pd.DataFrame(res.to_list())

,alt,definition,dict_word,info,leftovers,quantity
0,NaN,"Тогда, тотчас; теперь",АБИѤ,[нар.],И тоу абиѥ отъкрышѩ || ѥдинъ. отъ златыхъ онѣх...,857.0
1,NaN,"В знач. Так как, ибо",АБО,"[союз., причин.]",б҃иѥ бо сущьство не раздѣлѩѥть(с) ни преминуѥт...,1.0
2,NaN,В целевом знач. Чтобы,АБЫ,[союз.],"а кɤзеке соци абы не истьрѩле ɤно ГрБ № 163, 7...",91.0
3,[-Ы],"и ἄββα. Отец, настоятель монастыря, монах-старец",АВВ|А,"[нескл., с.]",Повѣдаша намъ авва. іѡанъ. и авва софронии гл҃...,283.0
4,[-А],"αὔγουστος Название месяца, август",АВГОУСТ|Ъ,[с.],м(с)цѩ авъгоуста въ к҃в҃ престависѩ рабъ б҃жии...,138.0
5,NaN,к авгоустъ,АВГОУСТЬСКЫИ,[пр.],Иоульска˫а лу(н). настане(т). иоунѩ. въ •е҃• а...,19.0
6,[-Ъ],"Αὐδιανοί Сторонники ереси, связанной с именем ...",АВДИ˫АН|Е,"[с., мн.]",о҃• Авди˫ане распьрѣниѥ покорениѥ. не оубо ере...,3.0
7,[-А],Древнееврейское название одного из летних месяцев,АВ|Ъ,[с.],"и наставшю •е҃-моу м(с)цю Авоу того(ж) лѣта, о...",2.0
8,NaN,NaN,АГГЕЛИТИ,NaN,NaN,NaN
9,NaN,NaN,АГГЕЛЪ,NaN,NaN,NaN


### Second old-russian dictitonary

In [ ]:
path = "chromedriver.exe"
driver = Chrome(path)
# driver.implicitly_wait(20)
driver.get('http://oldrusdict.ru/dict.html#')

In [ ]:
cols = (
    'Словарная статья',
    'ФП',
    'Производные формы',
    'Ссылки',
    'Значения',
    'Страница',
    'Часть речи'
)

srez = list()
# for char in "абвгдежзиклмнопрстуфхцчшщюяѣѳѵ":
for char in "абвгдежзиклмнопрстуфхцчшщъыьэюяѣѥѧѩѫѭѯѱѳѵї":
    inp_field = driver.find_element_by_id("word1")
    inp_field.clear()
    inp_field.send_keys(f"{char}*", Keys.ENTER)
    time.sleep(3)
    refs = driver.find_elements_by_xpath('//*[@id="words"]/table[2]/tbody/tr[53]/td/a')
    for i in range(len(refs)):
        pointer = i 
        if i != 0:
            pointer -= 1
        
        refs[pointer].click()
        time.sleep(3)
        records = driver.find_elements_by_xpath('//table[@cellpadding="5"]//tr[@valign="top"]')
        for record in records:
            cells = record.find_elements_by_tag_name("td")
            srez.append({col: cell.text for col, cell in zip(cols, cells)})
        
        refs = driver.find_elements_by_xpath('//*[@id="words"]/table[2]/tbody/tr[53]/td/a')
    with open(f"srez_{char}.pickle", "wb") as f:
        pickle.dump(srez, f)
driver.close()
srez = pd.DataFrame(srez)

In [ ]:
# with open("srez.pickle", "wb") as f:
#     pickle.dump(srez, f)

### Old-english dict

In [46]:
with open("ed.txt", "r", encoding='utf-8') as f:
    text = f.read()

text = text.split("\n\n")[1:]

text = [x.strip("\n") for x in text]

In [169]:
def parse_oed_line(line):
#     s = line.lower()
    s = line
    res = dict()
    finds = re.search(r"(?<=\+)[\w\-]+?(?=\+)", s)
    if finds is None:
        print(line)
        return None
    
    end = finds.span()[1]
    res["word"] = finds[0].lower()
    
    if re.match(r"\+,?\ ?_", s[end:]):
        finds = re.search(r"(?<=_)[\w\ \.]+?(?=_)", s)
        if finds:
            end = finds.span()[1]
            res["pos"] = finds[0].lower()
    
    for i, c in enumerate(s[end:]):
        if c.istitle() or (c == ";"):
            def_end = end + i
            break
    else:
        def_end = len(s)
        
    res["definition"] = re.sub(r"(?:[^\w\ ]|_)", "", s[end:def_end]).strip()
    
    return res


In [170]:
oed = list()

for i, line in enumerate(text):
    if line.startswith("+"):
        oed.append(parse_oed_line(line))


+Al-halowen day+, _sb._ All Saints’ Day, NED.--AS. _ealra halgena dæg_.
+Al solne day+, All Souls’ Day, NED; +alle soule day+, S2.--AS. _ealra
sawlena dæg_.
+Bi-long on+, _prep._ pertaining to, S.
+Ewe ardaunt+, _sb._ burning water, S2.--AF. _ewe_, _eue_, _aigue_; Lat.
_aquam_, see Academy, No. 459, p. 139; cp. Goth. _ahwa_.
+Menyng(e)+, _sb._ remembrance, mention, H.
+Menyng(e)+, _sb._ complaining, PP.
+Merueillous(e)+, _adj._ marvellous, C2, PP.--OF. _merveillos_.
+Sam . . . . sam+, _conj._ whether . . . . or, S.
+Segen on+, _v._ to make to sink, to cast down, S.--AS. _on-sǽgan_,
‘prosternere’ (Grein).


In [171]:
oed = pd.DataFrame(x for x in oed if x is not None)[~(oed.definition == "")]

In [178]:
# with open("oed.pickle", "wb") as f:
#     pickle.dump(oed, f)

### Old-english compendium

In [21]:
ex = "https://quod.lib.umich.edu/m/middle-english-dictionary/dictionary?page={}&per_page=100&q=%2A&search_field=hnf&sort=sequence+asc"
oec = list()

for i in tqdm_notebook(range(1, 562)):
    url = ex.format(i)
    soup = get_soup(url)
    for el in soup.find("div", {"id": "documents"}).find_all("div", {"class": "entry-panel panel"}):
        res = dict()
        res["word"] = el.find("a").text
        res["pos"] = el.find("span", {"class": "index-pos"}).text
        try:
            res["definition"] = el.find("div", {"class": "definition"}).text
        except AttributeError:
            pass
        oec.append(res)
    

In [ ]:
with open("oec.pickle", "wb") as f:
    pickle.dump(oec, f)

### The DICTIONARY of  OLD ENGLISH: A to I

In [2]:
path = "chromedriver.exe"
driver = Chrome(path)
driver.implicitly_wait(5)

In [ ]:
# login manually

In [16]:
ex = "https://tapor.library.utoronto.ca/doe/?E{}"
doe = list()
rep = list()

for i in tqdm_notebook(range(1, 18700)):
    num = str(i)
    num = "0" * (5 - len(num)) + num
    url = ex.format(num)
    
    driver.get(url)
#     if driver.find_element_by_id("doe-result-panel").text.startswith('{"error"'):
#         continue
    
    try:
        res = dict()
        res["word"] = driver.find_element_by_xpath('//*[@class="doe-hd"]').text
        res["pos"] = driver.find_element_by_xpath('//*[@class="doe-pos"]').text
        res["definition"] = driver.find_element_by_xpath('//*[@class="doe-sense"]').text
        doe.append(res)
    except:
        rep.append(url)
    


In [26]:
# with open("doe_1.pickle", "wb") as f:
#     pickle.dump(doe, f)

In [27]:
print(len(doe))

15161


In [28]:
print(len(rep))

3539
